# Yahoo Groups message scraper

This notebook provides code that will dowload messages from *private* Yahoo Groups. Another notebook `YahooScra.jpynb` uses [Selenium](http://selenium-python.readthedocs.io/). This notebook uses [reqeusts](http://docs.python-requests.org/en/latest/user/install/).

Another GitHub.com repository ([YahooGroups-Archiver](https://github.com/andrewferguson/YahooGroups-Archiver)) provided guidance related to accessing *private* Yahoo Groups. After logging into the private group you need to provide two pieces of information from the cookies:

> Cookie information can be found through the use of a plug-in for
> your web browser. (I use 'Cookie Manager' on FireFox, although
> there are many other options for FireFox and other browsers). The
> two cookies you are looking for are called Y and T, and they are 
> linked to the domain yahoo.com. Extract the data from these 
> cookies, and paste it into the appropriate variables... a cookie
> will expire after a certain amount of time, which varies between 
> computers. This means that you may have to re-fetch the Y and T 
> cookie data every few days, or you will not be able to archive 
> private groups. ([YahooGroups-Archiver](https://github.com/andrewferguson/YahooGroups-Archiver))

(Last tested April 21, 2018)

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
from bs4 import BeautifulSoup
import requests
import json
import datetime
import time
import os

In [ ]:
# Get groupname from user.
grp_name = input(prompt='What is the group name you seek to scrape?  ')
# Give grp_name a default if no input.
if grp_name == '':
    grp_name = 'concatenative'

In [ ]:
# These variables will take the cookies from 'Cookie Manager' discussed above as strings.
cookie_T = ''
cookie_Y = ''

# Define a list to log successful attempts.
fetch_log = []

# Iterate through the private group messages. The most recent group message number 
# can be found at: https://groups.yahoo.com/api/v1/groups/<groupname>/messages?count=10
# Parse the results to discern the most recent message number.
for i in range(1737,0,-1):
    fetch_log.append(
        ('{} Working on message {}.'.format(
            str(datetime.datetime.now()), str(i))))
    # For demonstration purposes use a generic groupsuch as 'concatenative'
    
    # Alternate code approach available.
    # s = requests.Session()
    # post = s.get('https://groups.yahoo.com/api/v1/groups/' + grp_name + '/messages/' + str(i) + '/', cookies={'T': cookie_T, 'Y': cookie_Y})

    post = requests.get(
        'https://groups.yahoo.com/api/v1/groups/' + grp_name + '/messages/' + str(i) + '/', 
        cookies={'T': cookie_T, 'Y': cookie_Y})

    # Yahoo Groups api returns JSON.
    post_parsed = json.loads(post.text)
    
    # The api result uses html in the messageBody; remove html tags from messageBody.
    try:
        soup = BeautifulSoup(post_parsed['ygData']['messageBody'], 'html.parser')
        soupstring = soup.get_text()
    # If no message found for that index; populate with message not found.
    except KeyError:
        soupstring = 'Post number {} was not found.'.format(str(i))
    
    # Save the message body as a .txt file.
    post_file = open(
        os.path.join(
            'msgs', ''.join(
                (grp_name + '_' + str(i) + r'_post.txt'))), 'w', encoding='utf-8')
    post_file.write(soupstring)
    post_file.close()
    
    # Save the api result as a .json file.
    json_file = open(
        os.path.join(
            'msgs', ''.join(
                (grp_name + '_' + str(i) + r'_json.json'))), 'w', encoding='utf-8')
    json_file.write(post.text)
    json_file.close()
    
    # Optionally pause to assist in avoiding CAPTCHA and other anti-robot features.
    time.sleep(.1)

In [ ]:
# Save fetch_log for later reference.
with open(grp_name + '_fetch_log_' + 
        str(datetime.datetime.now())[2:16].replace(" ", "-").replace(":","") + 
        '.log', mode='w') as logfile:
            print('This is the log of fetched messages file from {}'.format(
                str(datetime.datetime.now())), file=logfile)
            print('Yahoo Group name {}.'.format(grp_name), file=logfile)
            for fetch_line in fetch_log:
                print(fetch_line, file = logfile)
logfile.close

In [ ]:
# Get location of messages to compile.
folder_location = input(prompt='What is the location of messages (no input will default to msgs)?  ')
# Give grp_name a default if no input.
if folder_location == '':
    folder_location = 'msgs'

In [ ]:
# Build a dataset from the files created above.

# Define a list to log errors.
error_log  = []

# Define a list to hold structured data.
grandlist = []

# Iterate through the message files.
for i in range(33102,0,-1):
    try:
        work_file = open(
            os.path.join(
                folder_location, grp_name + '_' + str(i) + '_json.json'), 'r', encoding='utf-8')
        work_parse = json.loads(work_file.read())
        
        # Define list to hold current record.
        mylist = []
        mylist = [
            work_parse['ygData']['userId'],
            work_parse['ygData']['authorName'],
            work_parse['ygData']['subject'],
            work_parse['ygData']['postDate'],
            str(datetime.datetime.fromtimestamp(
                int(work_parse['ygData']['postDate'])).strftime('%Y-%m-%d %H:%M:%S')),
            work_parse['ygData']['msgId'],
            work_parse['ygData']['prevInTopic'],
            work_parse['ygData']['nextInTopic'],
            work_parse['ygData']['prevInTime'],
            work_parse['ygData']['nextInTime'],
            work_parse['ygData']['topicId'],
            work_parse['ygData']['numMessagesInTopic'],
            work_parse['ygData']['messageBody']]
        work_file.close()
        
        work_file = open(
            os.path.join(
                folder_location, grp_name + '_' + str(i) + '_post.txt'), 'r', encoding='utf-8')
        mylist.append(work_file.read())
        work_file.close()

        # Add the current observation to the structured data set.
        grandlist.append(mylist)
    except FileNotFoundError:
        # If file not found, provide output and log error.
        print('Message number ' + str(i) + ' - Not found.')
        error_log.append('FileNotFoundError. Message number {}.'.format(str(i)))
    except KeyError:
        # If any of the JSON keys (variables) not found, provide output and log error.
        print('Message number ' + str(i) + ' - KeyError.')
        error_log.append('KeyError. Message number '.format(str(i)))
    except OSError:
        # If OSError, provide output and log error.
        print('Message number ' + str(i) + ' - OSError.')
        error_log.append('OSError. Message number '.format(str(i)))

In [ ]:
# Save error_log for later reference.
with open(grp_name + '_err_log_' + 
        str(datetime.datetime.now())[2:16].replace(" ", "-").replace(":","") +
        '.log', mode='w') as logfile:
            print('This is the error log file from {}'.format(
                str(datetime.datetime.now())), file = logfile)
            print('Yahoo Group name {}.'.format(grp_name), file = logfile)
            for error_line in error_log:
                print(error_line, file = logfile)
logfile.close

In [ ]:
# Define column list (With message body)
col_list = ['userId','authName','subject','Unix','Date',
            'msgId','preInTpc','nxtInTpc','preInTime',
            'nxtInTime','topicId','MsgsInTopic','msgRaw','msgTxt']

# Put structured data into a Pandas dataframe.
grand_df = DataFrame(grandlist, columns=col_list)

In [ ]:
# Check results.
grand_df.head()

In [ ]:
# Save to CSV
grand_df.to_csv(grp_name + '_messages.csv')

In [ ]:
import re

def remove_comments(commented_text):
    # Remove comments from html
    pull_string = commented_text[commented_text.find('<!--'):commented_text.find('-->')+3]
    clean_string = commented_text.replace(pull_string,'')
    return(clean_string)

def remove_unicode(uni_txt):
    # Replace common Unicode chars with reasonable substitutes
    uni_txt = re.sub('[\u2022]', '-', uni_txt)
    uni_txt = re.sub('[\u2014]', '-', uni_txt)
    uni_txt = re.sub('[\u2013]', '-', uni_txt)
    uni_txt = re.sub('[\u0335]', '-', uni_txt)
    uni_txt = re.sub('[\u2011]', '-', uni_txt)
    uni_txt = re.sub('[\u1427]', '-', uni_txt)
    uni_txt = re.sub('[\u2027]', '-', uni_txt)
    uni_txt = re.sub('[\u2015]', '-', uni_txt)
    uni_txt = re.sub('[\u2212]', '-', uni_txt)
    uni_txt = re.sub('[\u25cf]', '-', uni_txt)
    uni_txt = re.sub('[\u25aa]', '-', uni_txt)
    uni_txt = re.sub('[\u00B7]', '-', uni_txt)
    
    uni_txt = re.sub('[\u201a]', "'", uni_txt)
    uni_txt = re.sub('[\u2019]', "'", uni_txt)
    uni_txt = re.sub('[\u2018]', "'", uni_txt)
    uni_txt = re.sub('[\u200b]', "'", uni_txt)
    uni_txt = re.sub('[\u2026]', "'", uni_txt)
    
    uni_txt = re.sub('[\u201c]', '"', uni_txt)
    uni_txt = re.sub('[\u201d]', '"', uni_txt)
    uni_txt = re.sub('[\u04cf]', "'", uni_txt)

    uni_txt = re.sub('[\u2002]', ' ', uni_txt)
    uni_txt = re.sub('[\ufddf]', ' ', uni_txt)
    uni_txt = re.sub('[\ufffd]', ' ', uni_txt)
    uni_txt = re.sub('[\U0001f603]', ' ', uni_txt)
    
    uni_txt = re.sub('[\u2028]', '... ...', uni_txt)
    uni_txt = re.sub('[\u263a]', ' smile ', uni_txt)
    
    uni_txt = re.sub('[\u25ba]', '>', uni_txt)
    uni_txt = re.sub('[\u2039]', '<', uni_txt)
    uni_txt = re.sub('[\u2329]', '<', uni_txt)
    uni_txt = re.sub('[\u232A]', '>', uni_txt)
    
    uni_txt = re.sub('[\u002D]', '-', uni_txt)
    uni_txt = re.sub('[\u03c0]', 'pi', uni_txt)
    uni_txt = re.sub('[\uf0d8]', '|?|', uni_txt)
    uni_txt = re.sub('[\u2502]', '|', uni_txt)
    uni_txt = re.sub('[\u0160]', 'S', uni_txt)
    uni_txt = re.sub('[\u02dc]', '~', uni_txt)
    uni_txt = re.sub('[\u0153]', 'oe', uni_txt)
    uni_txt = re.sub('[\ufb01]', 'fi', uni_txt)
    uni_txt = re.sub('[\u201e]', '"', uni_txt)
    uni_txt = re.sub('[\u0192]', 'f', uni_txt)
    uni_txt = re.sub('[\u2030]', '%', uni_txt)
    uni_txt = re.sub('[\u00AE]', '(r)', uni_txt)
    uni_txt = re.sub('[\u00BD]', '1/2', uni_txt)
    uni_txt = re.sub('[\u00A7]', 'Section', uni_txt)
    
    uni_txt = re.sub('[\u2010]', ' ', uni_txt)
    uni_txt = re.sub('[\ufffc]', ' ', uni_txt)
    uni_txt = re.sub('[\u200e]', ' ', uni_txt)
    uni_txt = re.sub('[\uf050]', ' ', uni_txt)
    uni_txt = re.sub('[\u25a1]', ' ', uni_txt)
    uni_txt = re.sub('[\uf097]', ' ', uni_txt)
    uni_txt = re.sub('[\u20AC]', ' ', uni_txt)
    uni_txt = re.sub('[\u2122]', ' ', uni_txt)
    
    uni_txt = re.sub('[\u2282]', '', uni_txt)
    uni_txt = re.sub('[\uf0a7]', '', uni_txt)
    uni_txt = re.sub('[\uf0a7]', '', uni_txt)
    uni_txt = re.sub('[\uf0a7]', '', uni_txt)
    uni_txt = re.sub('[\uf0a7]', '', uni_txt)
    uni_txt = re.sub('[\u2020]', '', uni_txt)
    uni_txt = re.sub('[\u00AC]', '', uni_txt)
    uni_txt = re.sub('[\u00B8]', '', uni_txt)
    uni_txt = re.sub('[\uF1FF]', '', uni_txt)
    uni_txt = re.sub('[\u3050]', '', uni_txt)
    uni_txt = re.sub('[\u98B5]', '', uni_txt)
    uni_txt = re.sub('[\u11CF]', '', uni_txt)
    uni_txt = re.sub('[\u82BB]', '', uni_txt)
    uni_txt = re.sub('[\uAA00]', '', uni_txt)
    uni_txt = re.sub('[\uBD00]', '', uni_txt)
    uni_txt = re.sub('[\u0bce]', '', uni_txt)
    uni_txt = re.sub('[\uf022]', '', uni_txt)
    uni_txt = re.sub('[\u221d]', '', uni_txt)
    uni_txt = re.sub('[\uf0a7]', '', uni_txt)
    uni_txt = re.sub('[\U0010003e]', '', uni_txt)
    
    # Remove leading and trailing spaces
    uni_txt = uni_txt.strip()
    
    # Remove other miscellaneous characters
    ret_txt = ''
    for item in uni_txt:
        ret_txt += item if len(item.encode(encoding='utf_8')) == 1 else ''
    
    # Remove <!--   --> style comments from html text
    ret_txt = remove_comments(ret_txt)
    
    return(ret_txt)

grand_df['msgRaw'] = grand_df['msgRaw'].apply(remove_unicode)
grand_df['msgTxt'] = grand_df['msgTxt'].apply(remove_unicode)
grand_df['subject'] = grand_df['subject'].apply(remove_unicode)

In [ ]:
# Save to Stata
# Problems on this. See: https://github.com/pandas-dev/pandas/issues/16450
grand_df.to_stata(grp_name + '_messages.dta', version=117, convert_strl=['subject','msgRaw','msgTxt'])

In [ ]:
# Save to Excel
writer = pd.ExcelWriter(grp_name + '_messages.xlsx', engine='xlsxwriter')
grand_df.to_excel(writer, sheet_name='Sheet1')
writer.save()